In [ ]:
data_dir = '../../../Kaggle_Data/Salt/results/models_4/'
result_files = [
    ('FPN_224_f0_84d1a7', 23,22,21,24,),
    ('FPN_224_f1_f5d73b', 24,22,21,20,),
    ('FPN_224_f2_0970dc', 24,11,19,21,),
    ('FPN_224_f3_efc38c', 22,20,23,11,)
   
]
thresholds = [ 0.5, 0.46, 0.63, 0.51 ]
submission_file_name = data_dir+'FPN_224_f0123all05'
save_results_dir = data_dir+'FPN_224_results/'
eval_crop_size = 224

In [ ]:
import numpy as np
import pandas as pd
import os
import load_data
from save_results import ResultsFileName, PredictResults

In [ ]:
test_df = load_data.LoadData(train_data = False, DEV_MODE_RANGE = 0)
test_images = test_df.images

In [ ]:
total = 0
for i, f_arr in enumerate(result_files):
    for j, x in enumerate(f_arr):
        if j == 0:
            params_file = x
        else:
            data_file = ResultsFileName(save_results_dir, params_file, x)
            if not os.path.exists(data_file):
                data_file += '.npy'
            if os.path.exists(data_file):
                print('loading '+ data_file)
                r = np.load(data_file)
            else:
                print('creating ' + data_file)
                r = PredictResults(test_images, data_dir, params_file, x, save_results_dir, eval_crop_size = 224)
            if total == 0:
                s = r
            else:
                s += r
            total += 1
            print('done ', total)
s /= total
    

In [ ]:
#threshold_best = np.mean(thresholds)
threshold_best = 0.5
threshold_best

In [ ]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
import time
t1 = time.time()
pred_dict = {idx: rle_encode((s[i] > threshold_best)) for i, idx in enumerate(test_df.index.values)}
t2 = time.time()


In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv(submission_file_name+'.submission.csv.gz', compression = 'gzip')